In [8]:
import csv
import urllib
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse
import random
import re

# Open the CSV file in read mode
with open('/Users/tahers/Documents/SE_NPM_packages/Dataset/random_packages_20k.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)

    # Handle potential header row
    is_header = True
    package_names = []
    for row in reader:
        if is_header:
            is_header = False  # Skip the header row if present
            continue
        package_names.append(row[0]) 

packages = package_names[11000:11020]
packages = ['is-equal-shallow', 'node-red-contrib-basic-thermostat', 'wintersmith']
print(len(packages))
output_filename = "synk_data_batch2.csv"

def time_duration_to_months(time_str):
    duration, unit, time = time_str.split()
    duration = float(duration)
    
    if unit == 'years' and time == 'ago':
        return round(duration * 12, 2)
    elif unit == 'months' and time == 'ago':
        return round(duration, 2)
    elif unit == 'days' and time == 'ago':
        return round(duration / 30, 2)
    else:
        raise time_str
    
def extract_size(value):
    try:
        match = re.search(r'([0-9.]+) *(B|kB|MB|GB)', str(value), re.IGNORECASE)
        if match:
            num_value = float(match.group(1)) 
            unit = match.group(2)
            if re.match(r'(B)', unit, re.IGNORECASE):
                return round(num_value / 1000, 2)
            elif re.match(r'(kB)', unit, re.IGNORECASE):
                return round(num_value, 2)
            elif re.match(r'(MB)', unit, re.IGNORECASE):
                return round(num_value * 1000, 2)
            elif re.match(r'(GB)', unit, re.IGNORECASE):
                return round(num_value * 1000000, 2)
        else:
            return np.NaN
    except:
        print(value)
        return value

def calculate_package_age(age):
    duration, unit = age.split()
    duration = float(duration)
    
    if unit == 'years':
        return round(duration * 12, 2)
    elif unit == 'months':
        return round(duration, 2)
    elif unit == 'days':
        return round(duration / 30, 2)
    else:
        raise age

def get_package_data(package):
    retry_attempts = 3  # Number of retry attempts
    for _ in range(retry_attempts):
        try:
            url = 'https://snyk.io/advisor/npm-package/{}'.format(urllib.parse.quote_plus(package))
            print("pkg:", package, "-", "GET", url)

            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            driver = webdriver.Chrome(options=options)
            driver.get(url)

            # Wait for the page to fully load
            time.sleep(0.38)

            # Extract HTML content after waiting
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, "html.parser")

            months_since_latest_version = license = health_score = security_recommendation = popularity = maintainence = community = age = age_months = versions = install_size = num_of_files = TS_typings = vulnerabilities_count = None


            #getting latest_version and license name
            container_elements = soup.find_all('div', class_='container')
            values_list = []

            for container in container_elements:
                values = container.find_all('div', class_='item')
                for value in values:
                    span = value.find('span')
                    if span:
                        values_list.append(span.text.strip())

            if values_list:
                latest_version = values_list[0].replace('Latest version published', '').strip()

                months_since_latest_version = time_duration_to_months(latest_version)
                
                license = values_list[1].replace('License: ', '').strip()

            #getting heath score
            number_div = soup.find('div', class_='number')
            if number_div:
                health_score = number_div.find('span').text.strip()[:3].rstrip()

            #getting and security and activeness metrices
            ul_element = soup.find('ul', class_='scores')
            if ul_element:
                spans = ul_element.find_all('span', class_='vue--pill__body')
                security_recommendation = spans[0].text.strip()
                popularity = spans[1].text.strip()
                maintainence = spans[2].text.strip()
                community = spans[3].text.strip()

            #getting package metrices
            stats_grid = soup.find_all('dl', class_='stats stats--grid')
            stats_grid_values = []
            for grid in stats_grid:
                grids = grid.find_all('span')
                stats_grid_values += [grid1.text.strip() for grid1 in grids]
            if stats_grid_values:
                age = stats_grid_values[11]
                age_months = calculate_package_age(age)
                versions = stats_grid_values[15]
                install_size = extract_size(stats_grid_values[17])
                num_of_files = stats_grid_values[21]
                TS_typings = stats_grid_values[25]

            # getting vulnerabilities_count
            table = soup.find('table', class_='vue--security-severity-table')
            vulnerabilities_count = []
            if table:
            
                highlighted_row = table.find('tr', class_='vue--security-severity-table__highlighted')
                if highlighted_row:
                
                    ul = highlighted_row.find('ul', class_='vue--severity')
                    if ul:
                        
                        li_tags = ul.find_all('li')
                        for li in li_tags:
                            
                            div_tags = li.find_all('div', class_='vue--severity__count')
                            for div in div_tags:
                                
                                span_tags = div.find_all('span')
                                for span in span_tags:
                                    vulnerabilities_count.append(span.text.strip())
            if vulnerabilities_count:
                critical_vul_count = int(vulnerabilities_count[0])
                high_vul_count = int(vulnerabilities_count[1])
                medium_vul_count = int(vulnerabilities_count[2])
                low_vul_count = int(vulnerabilities_count[3])
                vulnerabilities_count = critical_vul_count + high_vul_count + medium_vul_count + low_vul_count

            if TS_typings == 'Yes':
                TS_typings = True
            else:
                TS_typings = False

            driver.quit()

            if any(value == 'pending...' or value.strip() == '?' for value in [latest_version, license, health_score]):
                    print("Retrying for package:", package)
                    continue  # Retry for this package
            else:
                    return [package, months_since_latest_version, license, health_score, security_recommendation, popularity, maintainence, community, age_months, versions, install_size, num_of_files, TS_typings, vulnerabilities_count]

        except Exception as e:
            print("Error occurred while processing package:", package)
            print(e)
            return None

    print("Failed to retrieve data for package after", retry_attempts, "attempts:", package)
    return None

cnt=0
with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['package_name', 'months_since_latest_version_release', 'license', 'Health_score', 'Security_recommendation', 'Popularity', 'Maintainence', 'Community', 'package_age_months', 'num_of_versions', 'package_install_size', '#_of_files', 'TS_Typings', 'vulnerabilities_count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for package in packages:
        cnt+=1
        package_data = get_package_data(package)
        if package_data:
            writer.writerow(dict(zip(fieldnames, package_data)))
        print(cnt)
print("Data written to", output_filename)

3
pkg: is-equal-shallow - GET https://snyk.io/advisor/npm-package/is-equal-shallow
1
pkg: node-red-contrib-basic-thermostat - GET https://snyk.io/advisor/npm-package/node-red-contrib-basic-thermostat
2
pkg: wintersmith - GET https://snyk.io/advisor/npm-package/wintersmith
3
Data written to synk_data_batch2.csv
